# AutoGluon을 활용한 앙상블 모델 하이퍼파라미터 최적화
## California Housing Dataset

이 노트북에서는 AutoGluon의 자동 하이퍼파라미터 튜닝 기능을 사용하여
GBM, AdaBoost, XGBoost, LightGBM 모델을 최적화합니다.

### 필요한 라이브러리 설치

In [ ]:
# AutoGluon 설치 (처음 실행 시 필요)
!pip install autogluon

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# numpy 소수점 4째자리까지 표현
np.set_printoptions(precision=4)

### 데이터셋 가져오기

In [ ]:
from sklearn.datasets import fetch_california_housing

data = fetch_california_housing()

# 데이터프레임으로 변환
df = pd.DataFrame(data['data'], columns=data['feature_names'])
df['MedHouseVal'] = data['target']

print(f"데이터셋 크기: {df.shape}")
df.head()

In [ ]:
df.describe()

### 데이터 분할 (Train/Test Split)

In [ ]:
from sklearn.model_selection import train_test_split

# AutoGluon은 DataFrame 형태로 데이터를 받음
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Train set: {train_df.shape}")
print(f"Test set: {test_df.shape}")

---
## AutoGluon을 사용한 모델 최적화

AutoGluon은 자동 머신러닝(AutoML) 라이브러리로, 하이퍼파라미터 튜닝을 자동으로 수행합니다.

**사용할 모델:**
- **GBM (LightGBM)**: AutoGluon 내장 LightGBM
- **XGB (XGBoost)**: AutoGluon 내장 XGBoost  
- **CAT (CatBoost)**: AutoGluon 내장 CatBoost
- **RF (Random Forest)**: sklearn 기반 Random Forest

> 참고: sklearn의 GradientBoostingRegressor와 AdaBoostRegressor는 AutoGluon에서 직접 지원하지 않으므로,
> AutoGluon 학습 후 별도로 최적화합니다.

In [ ]:
from autogluon.tabular import TabularPredictor
from autogluon.common import space

### 하이퍼파라미터 검색 공간 정의

AutoGluon의 `space` 모듈을 사용하여 하이퍼파라미터 검색 범위를 지정합니다.

In [ ]:
# 모델별 하이퍼파라미터 검색 공간 정의
hyperparameters = {
    # LightGBM (GBM)
    'GBM': [
        {
            'num_boost_round': space.Int(100, 1000, default=500),
            'learning_rate': space.Real(0.01, 0.3, default=0.1, log=True),
            'num_leaves': space.Int(20, 100, default=31),
            'max_depth': space.Int(3, 10, default=6),
            'min_data_in_leaf': space.Int(10, 100, default=20),
            'feature_fraction': space.Real(0.5, 1.0, default=0.8),
            'bagging_fraction': space.Real(0.5, 1.0, default=0.8),
            'bagging_freq': space.Int(1, 10, default=5),
            'lambda_l1': space.Real(0.0, 1.0, default=0.0),
            'lambda_l2': space.Real(0.0, 1.0, default=0.0),
        }
    ],
    
    # XGBoost
    'XGB': [
        {
            'n_estimators': space.Int(100, 1000, default=500),
            'learning_rate': space.Real(0.01, 0.3, default=0.1, log=True),
            'max_depth': space.Int(3, 10, default=6),
            'min_child_weight': space.Int(1, 10, default=1),
            'subsample': space.Real(0.5, 1.0, default=0.8),
            'colsample_bytree': space.Real(0.5, 1.0, default=0.8),
            'gamma': space.Real(0, 1, default=0),
            'reg_alpha': space.Real(0, 1, default=0),
            'reg_lambda': space.Real(0, 1, default=1),
        }
    ],
    
    # CatBoost
    'CAT': [
        {
            'iterations': space.Int(100, 1000, default=500),
            'learning_rate': space.Real(0.01, 0.3, default=0.1, log=True),
            'depth': space.Int(4, 10, default=6),
            'l2_leaf_reg': space.Real(1, 10, default=3),
            'random_strength': space.Real(0, 1, default=0.5),
        }
    ],
    
    # Random Forest (배깅 기반)
    'RF': [
        {
            'n_estimators': space.Int(100, 500, default=300),
            'max_depth': space.Int(5, 20, default=None),
            'min_samples_split': space.Int(2, 20, default=2),
            'min_samples_leaf': space.Int(1, 10, default=1),
            'max_features': space.Real(0.5, 1.0, default=0.8),
        }
    ],
}

print("하이퍼파라미터 검색 공간이 정의되었습니다.")
print(f"튜닝할 모델: {list(hyperparameters.keys())}")

### AutoGluon 학습 및 하이퍼파라미터 튜닝

In [ ]:
# TabularPredictor 생성 및 학습
predictor = TabularPredictor(
    label='MedHouseVal',
    eval_metric='root_mean_squared_error',
    path='autogluon_models'
)

# 하이퍼파라미터 튜닝 설정
hyperparameter_tune_kwargs = {
    'num_trials': 30,           # 각 모델당 시도할 하이퍼파라미터 조합 수
    'scheduler': 'local',       # 로컬 스케줄러 사용
    'searcher': 'auto',         # 자동 탐색 알고리즘 선택
}

# 모델 학습
predictor.fit(
    train_data=train_df,
    hyperparameters=hyperparameters,
    hyperparameter_tune_kwargs=hyperparameter_tune_kwargs,
    time_limit=600,             # 최대 10분 (초 단위)
    presets='best_quality',     # 최고 품질 프리셋
    verbosity=2
)

### AutoGluon 결과 확인

In [ ]:
# 리더보드 확인
leaderboard = predictor.leaderboard(test_df, silent=True)
print("=" * 80)
print("AutoGluon 모델 리더보드")
print("=" * 80)
leaderboard

In [ ]:
# 최적 모델 정보
print(f"\n최적 모델: {predictor.model_best}")
print(f"\n모델별 성능 요약:")
print(predictor.fit_summary(show_plot=False))

In [ ]:
# 특성 중요도 확인
importance = predictor.feature_importance(test_df)
print("\n특성 중요도:")
importance

### 테스트 데이터 평가

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 예측
y_test = test_df['MedHouseVal']
y_pred = predictor.predict(test_df)

# 평가 지표 계산
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("=" * 50)
print("AutoGluon 최적 모델 평가 결과")
print("=" * 50)
print(f"MSE:  {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R2:   {r2:.4f}")

In [ ]:
# 예측값 vs 실제값 시각화
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5, edgecolors='k', linewidths=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
plt.xlabel('Actual Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.title(f'AutoGluon Best Model: Predicted vs Actual\n(RMSE: {rmse:.4f}, R2: {r2:.4f})', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

---
## sklearn GradientBoosting & AdaBoost 최적화 (Optuna 사용)

AutoGluon에서 직접 지원하지 않는 sklearn의 GradientBoostingRegressor와 AdaBoostRegressor는
Optuna를 사용하여 하이퍼파라미터 튜닝을 수행합니다.

In [ ]:
!pip install optuna -q

In [ ]:
import optuna
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import cross_val_score

optuna.logging.set_verbosity(optuna.logging.WARNING)

# sklearn용 데이터 분할
X_train = train_df.drop('MedHouseVal', axis=1)
y_train_sk = train_df['MedHouseVal']
X_test = test_df.drop('MedHouseVal', axis=1)
y_test_sk = test_df['MedHouseVal']

### GradientBoostingRegressor 최적화

In [ ]:
def objective_gbm(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'max_features': trial.suggest_float('max_features', 0.5, 1.0),
        'random_state': 42
    }
    
    model = GradientBoostingRegressor(**params)
    scores = cross_val_score(model, X_train, y_train_sk, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    return -scores.mean()

# GBM 최적화 실행
print("GradientBoostingRegressor 하이퍼파라미터 최적화 중...")
study_gbm = optuna.create_study(direction='minimize')
study_gbm.optimize(objective_gbm, n_trials=50, show_progress_bar=True)

print(f"\n최적 MSE: {study_gbm.best_value:.4f}")
print(f"최적 하이퍼파라미터: {study_gbm.best_params}")

In [ ]:
# 최적 GBM 모델 학습 및 평가
best_gbm = GradientBoostingRegressor(**study_gbm.best_params, random_state=42)
best_gbm.fit(X_train, y_train_sk)
y_pred_gbm = best_gbm.predict(X_test)

mse_gbm = mean_squared_error(y_test_sk, y_pred_gbm)
rmse_gbm = np.sqrt(mse_gbm)
r2_gbm = r2_score(y_test_sk, y_pred_gbm)

print("=" * 50)
print("최적화된 GradientBoostingRegressor 결과")
print("=" * 50)
print(f"MSE:  {mse_gbm:.4f}")
print(f"RMSE: {rmse_gbm:.4f}")
print(f"R2:   {r2_gbm:.4f}")

### AdaBoostRegressor 최적화

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def objective_ada(trial):
    base_max_depth = trial.suggest_int('base_max_depth', 2, 10)
    params = {
        'estimator': DecisionTreeRegressor(max_depth=base_max_depth, random_state=42),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 2.0, log=True),
        'loss': trial.suggest_categorical('loss', ['linear', 'square', 'exponential']),
        'random_state': 42
    }
    
    model = AdaBoostRegressor(**params)
    scores = cross_val_score(model, X_train, y_train_sk, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    return -scores.mean()

# AdaBoost 최적화 실행
print("AdaBoostRegressor 하이퍼파라미터 최적화 중...")
study_ada = optuna.create_study(direction='minimize')
study_ada.optimize(objective_ada, n_trials=50, show_progress_bar=True)

print(f"\n최적 MSE: {study_ada.best_value:.4f}")
print(f"최적 하이퍼파라미터: {study_ada.best_params}")

In [ ]:
# 최적 AdaBoost 모델 학습 및 평가
best_ada_params = study_ada.best_params.copy()
base_max_depth = best_ada_params.pop('base_max_depth')
best_ada = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(max_depth=base_max_depth, random_state=42),
    **best_ada_params,
    random_state=42
)
best_ada.fit(X_train, y_train_sk)
y_pred_ada = best_ada.predict(X_test)

mse_ada = mean_squared_error(y_test_sk, y_pred_ada)
rmse_ada = np.sqrt(mse_ada)
r2_ada = r2_score(y_test_sk, y_pred_ada)

print("=" * 50)
print("최적화된 AdaBoostRegressor 결과")
print("=" * 50)
print(f"MSE:  {mse_ada:.4f}")
print(f"RMSE: {rmse_ada:.4f}")
print(f"R2:   {r2_ada:.4f}")

---
## 전체 모델 성능 비교

In [ ]:
# 전체 결과 요약
results = {
    'Model': [],
    'MSE': [],
    'RMSE': [],
    'R2': []
}

# AutoGluon 모델들의 성능 추가
for model_name in leaderboard['model'].values:
    y_pred_model = predictor.predict(test_df, model=model_name)
    mse_model = mean_squared_error(y_test, y_pred_model)
    rmse_model = np.sqrt(mse_model)
    r2_model = r2_score(y_test, y_pred_model)
    
    results['Model'].append(f"AutoGluon_{model_name}")
    results['MSE'].append(mse_model)
    results['RMSE'].append(rmse_model)
    results['R2'].append(r2_model)

# sklearn 모델들 추가
results['Model'].append('Sklearn_GBM (Optuna)')
results['MSE'].append(mse_gbm)
results['RMSE'].append(rmse_gbm)
results['R2'].append(r2_gbm)

results['Model'].append('Sklearn_AdaBoost (Optuna)')
results['MSE'].append(mse_ada)
results['RMSE'].append(rmse_ada)
results['R2'].append(r2_ada)

# 결과 DataFrame 생성 및 정렬
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('RMSE').reset_index(drop=True)

print("=" * 80)
print("전체 모델 성능 비교 (RMSE 기준 정렬)")
print("=" * 80)
results_df

In [ ]:
# 성능 비교 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# RMSE 비교
colors = plt.cm.viridis(np.linspace(0, 0.8, len(results_df)))
axes[0].barh(results_df['Model'], results_df['RMSE'], color=colors)
axes[0].set_xlabel('RMSE', fontsize=12)
axes[0].set_title('Model Comparison by RMSE', fontsize=14)
axes[0].invert_yaxis()
for i, v in enumerate(results_df['RMSE']):
    axes[0].text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=10)

# R2 비교
axes[1].barh(results_df['Model'], results_df['R2'], color=colors)
axes[1].set_xlabel('R2 Score', fontsize=12)
axes[1].set_title('Model Comparison by R2 Score', fontsize=14)
axes[1].invert_yaxis()
for i, v in enumerate(results_df['R2']):
    axes[1].text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=10)

plt.tight_layout()
plt.show()

---
## 최적 하이퍼파라미터 요약

In [ ]:
print("=" * 80)
print("최적 하이퍼파라미터 요약")
print("=" * 80)

print("\n[1] sklearn GradientBoostingRegressor (Optuna 최적화):")
for key, value in study_gbm.best_params.items():
    print(f"    {key}: {value}")

print("\n[2] sklearn AdaBoostRegressor (Optuna 최적화):")
for key, value in study_ada.best_params.items():
    print(f"    {key}: {value}")

print("\n[3] AutoGluon 최적 모델 정보:")
print(f"    Best Model: {predictor.model_best}")

In [ ]:
# AutoGluon 모델별 하이퍼파라미터 확인
print("\n[AutoGluon 학습된 모델 정보]")
model_info = predictor.info()
print(f"모델 수: {model_info['model_info']['model_count']}")

---
## 결론

이 노트북에서는 AutoGluon과 Optuna를 사용하여 다양한 앙상블 모델의 하이퍼파라미터를 최적화했습니다.

**사용된 모델:**
- AutoGluon: LightGBM (GBM), XGBoost (XGB), CatBoost (CAT), Random Forest (RF)
- Optuna: sklearn GradientBoostingRegressor, AdaBoostRegressor

**주요 특징:**
1. AutoGluon은 자동으로 하이퍼파라미터 튜닝과 모델 앙상블을 수행
2. Optuna는 베이지안 최적화를 통해 효율적인 하이퍼파라미터 탐색 수행
3. 모든 모델의 성능을 비교하여 최적의 모델 선택 가능